# Remote Inference: Testing Your Deployed Model

This notebook demonstrates how to call your deployed Stable Diffusion model using the REST API.

https://kserve.github.io/website/latest/modelserving/data_plane/v1_protocol/

## Setup

Update the following variable to match your deployed model's *Inference endpoint*. For example: 

```
infer_endpoint = "https://redhat-dog-image-generation.apps.cluster.openshiftapps.com"
```

In [ ]:
infer_endpoint = "http://redhat-dog-predictor:8080"
infer_url = f"{infer_endpoint}/v1/models/model:predict"

## Request Function

Build and submit the REST request. The KServe V1 protocol expects JSON with an `instances` array.

In [ ]:
import requests

def rest_request(prompt):
    json_data = {
        "instances": [
            {
                "prompt": prompt,
                # "negative_prompt": "",
                # "num_inference_steps": 60,
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict

## Generate an Image

Test the deployed model with a prompt. Since we trained it with images of Teddy wearing his red fedora, mentioning the hat helps!

In [ ]:
# using "wearing his red fedora" because the model keeps forgetting his hat!
prompt = "a photo of rhteddy dog wearing his red fedora in the snowy mountains"
json_response = rest_request(prompt)

In [ ]:
import base64

img_str = json_response["predictions"][0]["image"]["b64"]
img_data = base64.b64decode(img_str)

## Decode and Display

The response contains a base64-encoded PNG image. Let's decode and display it.

In [ ]:
from PIL import Image
import io

image = Image.open(io.BytesIO(img_data))
image.save("teddy.png", format="PNG")
image

## Summary

You've successfully tested your deployed model! The model is now accessible via REST API and ready for integration into applications.

Your fine-tuned model can now generate images of Teddy in various scenarios when called through the API endpoint.